In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.ui import Select

import time
import os
import zipfile

import shutil
from glob import glob
import pandas as pd
from datetime import datetime

In [15]:
# Папка, где хранится трекер
download_dir = r"C:\Users\tatarikov\Desktop\R scripts\Daily report"

In [16]:
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options)

In [ ]:
# Здесь было подключение к конкретной ИРК

In [18]:
# ВВод логина
wait = WebDriverWait(driver, 10)
input_field = wait.until(EC.presence_of_element_located((
        By.CSS_SELECTOR, "#id_username"
    )))
input_field.send_keys("login")

In [19]:
# Ввод пароля
input_field = wait.until(EC.presence_of_element_located((
        By.CSS_SELECTOR, "#id_password"
    )))
input_field.send_keys("password")

In [20]:
button = driver.find_element(By.CSS_SELECTOR, "#submit-id-submit")
button.click()

In [ ]:
# Здесь был переход на вкладку с находками

In [22]:
# Находим поле ввода
input_field = driver.find_element(By.CSS_SELECTOR, "#wrap > div.container-fluid.main > main > query-tbl > table > thead > tr.ng-table-filters > th:nth-child(2) > div > input")

# Очистка поля 
input_field.clear()

In [23]:
import pandas as pd
from datetime import datetime
import os

# Установка рабочей директории
work_dir = r"C:\Users\tatarikov\Desktop\R scripts\Daily report"
os.chdir(work_dir)

# Получение сегодняшней даты в формате ДД.ММ.ГГГГ
today_str = datetime.today().strftime('%d.%m.%Y')

# Формирование имени файла
filename = f"Query list_{today_str}.xlsx"

# Чтение Excel-файла
df = pd.read_excel(filename, engine='openpyxl', skiprows=0)

# Вывод первых строк
print(df.head())


  Screening.ID   Subject.ID  Query.number       Event.name  \
0      602-014  602-014-233        757550  Re-signature IC   
1      600-005  600-005-186        757014        Screening   
2      600-005  600-005-186        757643        Screening   
3      604-014  604-014-204        757026        Screening   
4      604-005          NaN        661319        Screening   

                                   CRF Entity.name                   Day/Time  \
0  264_2_Re-signature informed consent        DMIC  19.03.2025 16:59:05 +0300   
1                     264_2_Urinalysis       URTIM  19.03.2025 16:34:08 +0300   
2                   264_2_Biochemistry        BCYN  19.03.2025 17:01:17 +0300   
3                     264_2_Urinalysis       URTIM  19.03.2025 16:35:01 +0300   
4             264_2_Lytic bone lesions    SSVISRCR  24.10.2024 12:05:48 +0300   

                              User   Status  
0  Aleksandra Vasileva (vasilevaa)  Updated  
1  Aleksandra Vasileva (vasilevaa)  Updated  
2 

In [24]:
# Фильтрация по трём условиям:
# - Status == "Updated"
# - User == "Aleksandra Vasileva (vasilevaa)"
# - Day/Time начинается с "31.03.2025"

filtered_df = df[
    (df['Status'] == 'Updated') &
    (df['User'] == 'Aleksandra Vasileva (vasilevaa)') &
    (df['Day/Time'].astype(str).str.startswith('31.03.2025'))
]

# Вывод результата
print(filtered_df)


    Screening.ID   Subject.ID  Query.number              Event.name  \
9        608-003          NaN        762991               Screening   
13       600-001          NaN        762800               Screening   
21       602-011  602-011-179        762945               Screening   
23       600-004  600-004-157        763275               Screening   
29       609-010  609-010-210        763285               Screening   
30       606-003  606-003-164        762862               Screening   
32       606-015          NaN        762864               Screening   
33       606-015          NaN        763096               Screening   
42       606-001  606-001-089        762860               Screening   
45       606-001  606-001-089        763092               Screening   
46       606-001  606-001-089        763232               Screening   
49       608-002          NaN        762990               Screening   
50       608-001          NaN        762989               Screening   
51    

In [25]:
# Вывод количества строк
print(len(filtered_df))

53


In [13]:
# === Обработка каждой строки ===
for index, row in filtered_df.iterrows():
    query_number = row["Query.number"]

    # Ввод значения Query.number
    input_field = driver.find_element(By.CSS_SELECTOR,
        "#wrap > div.container-fluid.main > main > query-tbl > table > thead > tr.ng-table-filters > th:nth-child(4) input")
    
    input_field.clear()
    input_field.send_keys(str(query_number))
    input_field.send_keys(Keys.ENTER)
    time.sleep(2)

    # Проверка значения в 10-й колонке таблицы
    try:
        yes_no_field = driver.find_element(By.CSS_SELECTOR,
            "#wrap > div.container-fluid.main > main > query-tbl > table > tbody > tr > td:nth-child(10)")
        yes_no_value = yes_no_field.text.strip()
        print(f"[{query_number}] Значение поля Yes/No:", yes_no_value)
    except Exception as e:
        print(f"[{query_number}] Ошибка при проверке поля Yes/No:", e)
        continue

    if yes_no_value == "Yes":
        try:
            # Сначала читаем значение поля Type (5-я колонка)
            type_field = driver.find_element(By.CSS_SELECTOR,
                "#wrap > div.container-fluid.main > main > query-tbl > table > tbody > tr > td:nth-child(5)")
            type_value = type_field.text.strip()
            print(f"[{query_number}] Тип:", type_value)

            # Нажимаем кнопку действия (всегда)
            action_button = driver.find_element(By.CSS_SELECTOR,
                "#wrap > div.container-fluid.main > main > query-tbl > table > tbody > tr > td:nth-child(13) > span:nth-child(3) > button")
            action_button.click()
            time.sleep(2)

            if type_value == "Query monitoring data":
                # Ждём, пока обёртка с селектом появится
                description_wrapper = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.col-sm-9[ng-if='qrAeVm.edit']")))
                # Теперь можно искать сам селект
                select_element = description_wrapper.find_element(By.NAME, "description")

                # Используем Select

                select = Select(select_element)
                select.select_by_visible_text("Query response monitored")


                print(f"[{query_number}] Выбрано: Query response monitored.")

            elif type_value == "Reason for change":
                print(f"[{query_number}] Тип: Reason for change. Только сохранение.")

            else:
                print(f"[{query_number}] Необрабатываемый тип: {type_value}. Пропущено.")
                continue  # Не продолжаем сохранение

            # Нажимаем кнопку Save
            save_button = driver.find_element(By.CSS_SELECTOR,
                "body > div.modal.fade.in > div > div > div.modal-footer > button.btn.btn-primary")
            save_button.click()
            print(f"[{query_number}] Сохранение выполнено.")

            # Обновление значения в датафрейме
            filtered_df.at[index, 'Status'] = 'Closed'

        except Exception as e:
            print(f"[{query_number}] Ошибка при обработке:", e)

    else:
        print(f"[{query_number}] Поле равно 'No'. Пропускаем изменение.")
    
    time.sleep(2)

[763399] Значение поля Yes/No: Yes
[763399] Тип: Query monitoring data
[763399] Выбрано: Query response monitored.
[763399] Сохранение выполнено.
[763201] Значение поля Yes/No: Yes
[763201] Тип: Query monitoring data
[763201] Выбрано: Query response monitored.
[763201] Сохранение выполнено.
[763302] Значение поля Yes/No: Yes
[763302] Тип: Query monitoring data
[763302] Выбрано: Query response monitored.
[763302] Сохранение выполнено.
[762942] Значение поля Yes/No: Yes
[762942] Тип: Query monitoring data
[762942] Выбрано: Query response monitored.
[762942] Сохранение выполнено.
[763067] Значение поля Yes/No: Yes
[763067] Тип: Query monitoring data
[763067] Выбрано: Query response monitored.
[763067] Сохранение выполнено.
[762836] Значение поля Yes/No: Yes
[762836] Тип: Query monitoring data
[762836] Выбрано: Query response monitored.
[762836] Сохранение выполнено.
[763470] Значение поля Yes/No: Yes
[763470] Тип: Query monitoring data
[763470] Выбрано: Query response monitored.
[763470] 

In [26]:
# Список кверей в статусе No/NA
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

not_yes_rows = []  # Сюда будем складывать строки, у которых значение не "Yes"

# === Обработка каждой строки ===
for index, row in filtered_df.iterrows():
    query_number = row["Query.number"]

    try:
        # Ввод значения Query.number
        input_field = driver.find_element(By.CSS_SELECTOR,
            "#wrap > div.container-fluid.main > main > query-tbl > table > thead > tr.ng-table-filters > th:nth-child(4) input")
        
        input_field.clear()
        input_field.send_keys(str(query_number))
        input_field.send_keys(Keys.ENTER)
        time.sleep(2)

        # Проверка значения в 10-й колонке таблицы
        yes_no_field = driver.find_element(By.CSS_SELECTOR,
            "#wrap > div.container-fluid.main > main > query-tbl > table > tbody > tr > td:nth-child(10)")
        yes_no_value = yes_no_field.text.strip()
        print(f"[{query_number}] Значение поля Yes/No: {yes_no_value}")

        # Если значение НЕ "Yes", добавляем строку в список
        if yes_no_value != "Yes":
            not_yes_rows.append(row)

    except Exception as e:
        print(f"[{query_number}] Ошибка при получении значения Yes/No:", e)
        continue

    time.sleep(1)

# Создание нового датафрейма
not_yes_df = pd.DataFrame(not_yes_rows)

# Сохранение в Excel
not_yes_df.to_excel("not_yes_queries.xlsx", index=False)
print("Файл not_yes_queries.xlsx сохранён.")


[762991] Значение поля Yes/No: No
[762800] Значение поля Yes/No: No
[762945] Значение поля Yes/No: Yes
[763275] Значение поля Yes/No: No
[763285] Значение поля Yes/No: No
[762862] Значение поля Yes/No: Yes
[762864] Значение поля Yes/No: Yes
[763096] Значение поля Yes/No: Yes
[762860] Значение поля Yes/No: No
[763092] Значение поля Yes/No: No
[763232] Значение поля Yes/No: No
[762990] Значение поля Yes/No: No
[762989] Значение поля Yes/No: No
[763371] Значение поля Yes/No: No
[762922] Значение поля Yes/No: No
[763029] Значение поля Yes/No: No
[762861] Значение поля Yes/No: Yes
[763093] Значение поля Yes/No: No
[762875] Значение поля Yes/No: No
[763109] Значение поля Yes/No: No
[763049] Значение поля Yes/No: No
[763463] Значение поля Yes/No: No
[763407] Значение поля Yes/No: No
[763051] Значение поля Yes/No: No
[763258] Значение поля Yes/No: No
[762992] Значение поля Yes/No: No
[763457] Значение поля Yes/No: No
[762799] Значение поля Yes/No: No
[763349] Значение поля Yes/No: No
[762933] 